## 故事描述
水的气态、液态、固态三态变化

In [17]:
# 框架模型
class Context:
    "状态模式的上下文环境类"
    def __init__(self):
        self.__states = []
        self.__curState = None
        # 状态发生变化依赖的信息数据,在有多个变量决定状态的
        # 实际复杂应用场景中，可以将其单独定义成一个类
        self.__stateInfo = 0

    def addState(self, state):
        if (state not in self.__states):
            self.__states.append(state)
            
    def isSameState(self, state):
        if (self.__curState == state):
            return True
        else:
            return False

    def changeState(self, state):
        if (state is None):
            return False
        if (self.__curState is None):
            print("初始化为", state.getStateName())
        else:
            print("由", self.__curState.getStateName(), "变为", state.getStateName())
        self.__curState = state
        self.addState(state)
        return True

    def getState(self):
        return self.__curState

    def _setStateInfo(self, stateInfo):
        self.__stateInfo = stateInfo
        for state in self.__states:
            if( state.isMatch(stateInfo) ):
                self.changeState(state)

    def _getStateInfo(self):
        return self.__stateInfo


class State:
    "状态的基类"
    def __init__(self, name):
        self.__name = name

    def getStateName(self):
        return self.__name

    def isMatch(self, stateInfo):
        "状态信息stateInfo是否在当前的状态范围内"
        return False

    def behavior(self, context):
        pass

![avatar](images/03_1.png)
<center>图1 - 类图</center>

In [18]:
class Water(Context):
    "水(H2O)"

    def __init__(self):
        super().__init__()
        self.addState(SolidState("固态"))
        self.addState(LiquidState("液态"))
        self.addState(GaseousState("气态"))
        self.setTemperature(25)

    def getTemperature(self):
        return self._getStateInfo()

    def setTemperature(self, temperature):
        self._setStateInfo(temperature)

    def riseTemperature(self, step):
        self.setTemperature(self.getTemperature() + step)

    def reduceTemperature(self, step):
        self.setTemperature(self.getTemperature() - step)

    def behavior(self):
        state = self.getState()
        if(isinstance(state, State)):
            state.behavior(self)


# 单例的装饰器
def singleton(cls, *args, **kwargs):
    "构造一个单例的装饰器"
    instance = {}

    def __singleton(*args, **kwargs):
        if cls not in instance:
            instance[cls] = cls(*args, **kwargs)
        return instance[cls]

    return __singleton


@singleton
class SolidState(State):
    "固态"

    def __init__(self, name):
        super().__init__(name)

    def isMatch(self, stateInfo):
        return stateInfo < 0

    def behavior(self, context):
        if (isinstance(context, Water)):
            print("固态，当前温度", context.getTemperature(),"摄氏度。")


@singleton
class LiquidState(State):
    "液态"

    def __init__(self, name):
        super().__init__(name)

    def isMatch(self, stateInfo):
        return (stateInfo >= 0 and stateInfo < 100)

    def behavior(self, context):
        if (isinstance(context, Water)):
            print("液态，当前温度", context.getTemperature(),"摄氏度。")

@singleton
class GaseousState(State):
    "气态"

    def __init__(self, name):
        super().__init__(name)

    def isMatch(self, stateInfo):
        return stateInfo >= 100

    def behavior(self, context):
        if (isinstance(context, Water)):
            print("气态，当前温度", context.getTemperature(),"摄氏度。")

In [19]:
def testState():
    "状态模式的测试代码"
    water = Water()
    water.behavior()
    water.setTemperature(-4)
    water.behavior()
    water.riseTemperature(18)
    water.behavior()
    water.riseTemperature(18)
    water.behavior()
    water.riseTemperature(110)
    water.behavior()
    water.setTemperature(60)
    water.behavior()
    water.reduceTemperature(80)
    water.behavior()

In [20]:
testState()

初始化为 液态
液态，当前温度 25 摄氏度。
由 液态 变为 固态
固态，当前温度 -4 摄氏度。
由 固态 变为 液态
液态，当前温度 14 摄氏度。
液态，当前温度 32 摄氏度。
由 液态 变为 气态
气态，当前温度 142 摄氏度。
由 气态 变为 液态
液态，当前温度 60 摄氏度。
由 液态 变为 固态
固态，当前温度 -20 摄氏度。


## 模型说明
1. 在状态模式实现的时候，实现的场景状态有时候会非常复杂。决定状态变化的因素也会非常多，这个时候我们可以把决定状态变化的属性单独抽象成一个类 StateInfo，这样判断状态属性是否符合当前的状态 isMatch 时就可以传入更多的信息。
2. 每一种状态应当只有唯一的一个实例。

## 应用场景
1. 一个对象的行为取决于它的状态，并且它在运行时可能经常改变它的状态从而改变它的行为。
2. 一个操作中含有庞大的多分支的条件语句，且这些分支依赖于该对象的状态，且每一个分支的业务逻辑非常复杂时，我们可以使用状态模式来拆分他不同分支逻辑，使程序有更好的可读性可维护性。
